## **PROJECT-4.  Решение комплексной бизнес-задачи**
<font size="5" color="red"><b>DAPR-1016</b> Pidenko Dmitry</font>

**<font size="5" color="green"><b>ЛЕГЕНДА</b></font>**

     Итак, я работаю аналитиком в онлайн-школе MasterMind.

Уже в конце рабочего дня мне пишет расстроенный продакт-менеджер. Несчастный Григорий крайне устал от того, что новые курсы, созданные с той же любовью, что и прежние, не пользуются особой популярностью среди пользователей — несмотря на все усилия отдела маркетинга.


**<font size="5" color="green"><b>ЦЕЛЬ</b></font>**

Подготовить основу рекомендательной системы.

**<font size="5" color="green"><b>ЗАДАЧИ</b></font>**

Итак, продакт ожидает получить от меня рекомендательную систему, благодаря которой можно будет предлагать клиентам интересные им курсы и тем самым повышать средний чек.

Я решаю, что изначальным воплощением этой системы может стать таблица, в которой курсам будет соответствовать по две рекомендации.

Кроме того, я планирую вместе с отчётом (таблицей рекомендаций) скинуть продакту ещё и все написанные в процессе скрипты, чтобы было меньше вопросов по решению :)

Также я понимаю, что перед внедрением фичи коллеги решат провести A/B-тест и вас скорее всего привлекут к анализу результатов.

Перспективы ясны, можно переходить к формализации задач.



**<font size="5" color="green"><b>КОНКРЕТНЫЕ ШАГИ (ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ)</b></font>**

Обдумав план предстоящей работы, вы понимаете, что действовать нужно по привычной схеме:

1. Познакомиться с датасетом, подготовить и проанализировать данные с помощью SQL.

2. Обработать данные средствами Python.

3. Составить итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями, и представить отчёт продакт-менеджеру.

4. Проанализировать результаты A/B-теста, проведённого после внедрения фичи, и сделать вывод.

## **1.1.Изучаем данные**

У нас в распоряжении две таблицы:

<font size="5" color="green"><b>carts</b></font> с данными о пользовательских корзинах (дате создания, статусе, id пользователя-владельца и т. д.);

<font size="5" color="green"><b>cart items</b></font> с данными о курсах, которые пользователи добавили в корзину.

Необходимые таблицы находятся в схеме final

Располагаем данными о продажах за период от <font size="5" color="green"><b>2017</b></font> по <font size="5" color="green"><b>2018</b></font> год.

Общее количество уникальных пользователей которые произвели покупку курсов, составляет: <font size="5" color="green"><b>49006</b></font> человек

Общее количество уникальных курсов: <font size="5" color="green"><b>127</b></font>  штук

Среднее число купленных курсов на одного клиента составляет: <font size="5" color="green"><b>1.44</b></font> штук

Уникальное количество клиентов которые купили более одного курса: <font size="5" color="green"><b>12656</b></font> человек



## **1.2.Готовим данные для анализа в Metabase**

Сформируем  SQL-запрос с данными по продажам курсов в разрезе пользователей (купивших более одного курса) с помощью инструмента Metabase

<font size="1" color="green"><b>
WITH CTN_clients_1 AS

(

SELECT

DISTINCT c.user_id,

COUNT(DISTINCT s.resource_id)

FROM final.carts c

JOIN final.cart_items s ON c.id = s.cart_id

WHERE s.resource_type = 'Course' AND c.state = 'successful'

GROUP BY 1

HAVING COUNT(DISTINCT s.resource_id) > 1),

CTN_clients_2 AS

(

SELECT

*

FROM final.carts c

JOIN final.cart_items s ON c.id = s.cart_id

WHERE s.resource_type = 'Course' AND c.state = 'successful')

SELECT

DISTINCT i.user_id,

j.resource_id

FROM CTN_clients_1 i

JOIN CTN_clients_2 j ON i.user_id = j.user_id</b></font>

## **1.3.Произведем выгрузку необходимых библиотек**

In [2]:
import pandas as pd
import psycopg2
import psycopg2.extras
import numpy as np
import itertools
from itertools import combinations
from collections import Counter

## **1.4.Перейдём к анализу полученного списка с помощью Python**

Для того чтобы отразить таблицу в Jupyter Notebook, сформированную в Metabase ранее при помощи SQL- запроса. Необходимо воспользоваться следующей функцией, которая представлена ниже. Далее преобразуем данные в таблицу

In [3]:
query = '''
   WITH CTN_clients_1 AS
(
SELECT
      DISTINCT c.user_id,
      COUNT(DISTINCT s.resource_id)
FROM final.carts c
JOIN final.cart_items s ON c.id = s.cart_id
WHERE s.resource_type = 'Course' AND c.state = 'successful'
GROUP BY 1
HAVING COUNT(DISTINCT s.resource_id) > 1),
CTN_clients_2 AS
(
SELECT*
FROM final.carts c
JOIN final.cart_items s ON c.id = s.cart_id
WHERE s.resource_type = 'Course' AND c.state = 'successful')

SELECT
      DISTINCT i.user_id,
      j.resource_id
FROM CTN_clients_1 i
JOIN CTN_clients_2 j ON i.user_id = j.user_id
    '''.format()


def get_data():
    query
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data


df = pd.DataFrame(get_data()) #преобразование полученных данных в таблицу


In [4]:
#смотрим, что получилось
df.head()

,user_id,resource_id
0,909757,356
1,583850,515
2,1559882,566
3,970967,679
4,1640443,566


## **1.5.Готовим данные для анализа в Python**

Сгруппируем  курсы по пользователям и разобъем их внутри  по парам, а также не забудем избавиться от зеркальных пар с помощью сортировки. Нам нужны исключительно уникальные значения.


In [5]:
df_piv = df.groupby('user_id')['resource_id'].apply((lambda x:list(np.unique(x)))).reset_index()
#выведем 5 строк и посмотрим на результат
df_piv.head()

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"


Разабъем все покупки курсов по парам

In [6]:
df_piv_couser= df_piv.copy()
df_piv_couser= df_piv['resource_id'].apply(lambda x: list(combinations(x, 2)))
df_piv_couser = df_piv_couser.to_frame()
display(df_piv_couser.head())

,resource_id
0,"[(516, 1099)]"
1,"[(356, 357), (356, 1125), (357, 1125)]"
2,"[(553, 1147)]"
3,"[(361, 1138)]"
4,"[(356, 357)]"


Посчитаем количество различных пар курсов, которые встречаются вместе в покупках клиентов

In [7]:
courses_list = []
for course in df_piv_couser['resource_id']:
    for elem in course: 
        courses_list.append(elem) 
print("Количество различных пар курсов которые встречаются вместе в покупках клиентов:", len(set(courses_list)))

Количество различных пар курсов которые встречаются вместе в покупках клиентов: 3989


Проранжируем полученные пары по количеству раз совместной встречаемости с помощью модуля Counter

In [8]:
#Преобразование в словарь для последующей обработки
pair_cnt = Counter(courses_list)
course_dict =dict(pair_cnt.most_common()) 

1.Реализуем функцию подбора рекомендуемых курсов, которая на вход принимает один номер курса и выдает 2 (других) наиболее часто встречающихся с этим курсом

2.При помощи полученной функции сделать итерацию уникального списка всех курсов для получения 2-х доп рекомендаций к каждому(курсу)

In [9]:
#Преобразование проранжированных ранее пар курсов в таблицу (для последующего анализа)
courses_table = pd.DataFrame(pair_cnt.items(), columns=['course_pair', 'rank']).sort_values(by='rank', ascending=False)
display(courses_table.head()) 

,course_pair,rank
186,"(551, 566)",797
381,"(515, 551)",417
205,"(489, 551)",311
105,"(523, 551)",304
519,"(566, 794)",290


Создадим список уникальных идентификаторов курсов (для итерации в результирующей таблице)

In [10]:
uniq_list =df['resource_id'].unique()
display(uniq_list)

array([ 356,  515,  566,  679,  504,  551,  570,  514,  571, 1129,  503,
        523,  568,  511, 1138,  745,  516,  741, 1181,  569,  764,  572,
        794,  490,  489,  564,  502,  753,  750,  829,  791,  752,  776,
        563,  908, 1144, 1128,  357, 1141,  553,  519,  365,  659,  368,
       1125,  840,  863,  358,  809, 1103,  363, 1184,  756,  360,  507,
       1100,  552,  664,  518,  359,  757,  672,  765, 1156, 1140,  562,
       1188,  509,  749,  366,  907, 1102, 1152, 1161,  517,  777, 1139,
        670, 1145,  671, 1101,  361, 1099,  513,  902,  743, 1115, 1116,
        367, 1187,  912,  814, 1124, 1186,  742,  909, 1146,  744,  508,
        835,  837,  364,  362,  803,  810,  813,  862, 1160,  800, 1104,
        755,  864, 1147,  834, 1185,  836, 1198, 1199,  911,  830,  833,
       1182,  865,  866, 1200, 1201], dtype=int64)

найдем общее количество уникальных курсов

In [11]:
course_cnt =Counter(df['resource_id'].unique())
print("Общее количество уникальных курсов:", sum(course_cnt.values()))

Общее количество уникальных курсов: 126


Найдем ID самой популярной группы курсов

In [12]:
popular_group = Counter(courses_list).most_common()
print('Самая популярная пара курсов:', popular_group[0])

Самая популярная пара курсов: ((551, 566), 797)


**<font size="5" color="green"><b>ЗАДАНИЕ</b></font>**

Необходимо оформить таблицу, которая станет основой рекомендательной системы.

Прежде всего нужно установить минимальную границу — какое количество раз считать слишком малым.
Вместо малопопулярного курса вывести какой-то другой курс, который, на ваш аргументированный взгляд, подходит лучше.

Итоговая таблица должна состоять из трёх столбцов:

    1.Курс, к которому идёт рекомендация
    2.Курс для рекомендации № 1 (самый популярный)
    3.Курс для рекомендации № 2 (второй по популярности)

Что делать, если одна из рекомендаций встречается слишком редко?

    Прежде всего нужно установить минимальную границу — какое количество раз считать слишком малым.

    Вместо малопопулярного курса вывести какой-то другой курс, который, на ваш аргументированный взгляд, подходит лучше.

## **2.1.Создадим новый датафрейм, а затем перегруппируемся**

In [13]:
#создадим дата фрейм курсов с частотой покупки
df2 = pd.DataFrame(popular_group,columns=['pair', 'freq'])
display(df2.head())

,pair,freq
0,"(551, 566)",797
1,"(515, 551)",417
2,"(489, 551)",311
3,"(523, 551)",304
4,"(566, 794)",290


Построим сводную таблицу по частоте пар курсов, чтобы увидеть, сколько пар встречаются один раз, два, раза раза и т.д.

In [14]:
#группируем пары курсов по частоте, определяем количество пар с такой частотой
pivot = df2.groupby(by = 'freq')['pair'].count()
pivot.head()

freq
1    1185
2     578
3     366
4     259
5     202
Name: pair, dtype: int64


**<font size="5" color="green"><b>ВЫВОД</b></font>**

Взглянув на сводную таблицу не вооруженным глазом множно увидеть, что из <font size="5" color="green"><b>3989</b></font> различных пар курсов, которые покупают пользователи, на каждую из <font size="5" color="green"><b>1185</b></font> пар курсов (30% от всех уникальных пар курсов) приходится одна покупка, на <font size="5" color="green"><b>578</b></font> пар (14% от всех уникальных пар курсов) - 2 покупки, <font size="5" color="green"><b>366</b></font> (9%) - 3 покупки и т.д. Самая популярная пара курсов <font size="5" color="green"><b>(551, 566)</b></font> была куплена <font size="5" color="green"><b>797</b></font> раз.

## **2.2.Посмотрим, как распределены частоты покупок курсов при помощи функции .describe()**

In [15]:
display(df2['freq'].describe())

count    3989.000000
mean       10.031838
std        26.355998
min         1.000000
25%         1.000000
50%         3.000000
75%         9.000000
max       797.000000
Name: freq, dtype: float64

**<font size="5" color="green"><b>ВЫВОД</b></font>**

Видим, что средняя частота покупок ~ 10%, где частота покупок больше 9%, следовательно исходя из этого, установим минимальную границу для формирования рекомендательной таблицы на уровен 9 покупок.

## **2.3.Сформируем новый датафрем и сгруппируемся по ID курсов, для того чтобы увидеть сколько раз продавался курс с определенном номером**

In [16]:
#проверим, какие курсы были самыми популярными и отсортируемся от большего к меньшему
popular_course = df.groupby(by = 'resource_id')['user_id'].count().sort_values(ascending=False)
popular_course.head()


resource_id
551    2935
566    2342
515    1311
489    1125
490    1084
Name: user_id, dtype: int64

**<font size="5" color="green"><b>ВЫВОД</b></font>**

Самый популярный курс <font size="5" color="green"><b>551</b></font>. По этому будем рекомендовать его к тем курсам, у которых частота покупки меньше 9%

## **3.1. Оформим таблицу, которая станет основой рекомендательной системы**

### <font size="5" color="green"><b>**Построение рекомендательной таблицы будет состоять из 2-х основных частей:**</b></font>

1. Напишем функцию рекомендаций(на вход будет подаваться номер курса, возвращаться будет 2 курса, с которым поданный на вход курс чаще всего продавался).

In [17]:
def course_recommendation(resource_id):
    recomm_list = [] 
    for i in course_dict.keys():
        if i[0] == resource_id: 
            recomm_list.append((i, course_dict[i]))
        elif i[1] == resource_id:
            recomm_list.append((i, course_dict[i]))
    recomm_list_sort =sorted(recomm_list,key = lambda x: x[1])
    return recomm_list[:2] 

2.Создадим цикл, который выводит список рекомендаций для каждого курса (на основе вышенаписанной функции).

 Отсутствующие рекомендации заполняем курсами из списка самых популярных [516, 566].



In [18]:
recommendation_list= []
recomm_df = pd.DataFrame(recommendation_list, columns=['first_rec', 'second_rec'])

for resource_id in uniq_list:
    rec1= None
    rec2= None
    if course_recommendation(resource_id)[0][1] >= 9:
        rec1 = (set(course_recommendation(resource_id)[0][0])-set([resource_id])).pop()
    if course_recommendation(resource_id)[1][1] >= 9:
        rec2 = (set(course_recommendation(resource_id)[1][0])-set([resource_id])).pop()
    recomm_df.loc[resource_id] = [rec1, rec2]
    
    if recomm_df.loc[resource_id][0] is None and recomm_df.loc[resource_id][1] is None:
        recomm_df.loc[resource_id] = [516, 566] 
    if rec2 is None and rec1 is not None:
        rec1 = (set(course_recommendation(resource_id)[0][0])-set([resource_id])).pop()
        rec2 = np.random.choice(list(set([516, 566])-set([resource_id])))
        recomm_df.loc[resource_id] = [rec1, rec2]

In [20]:
display(recomm_df.head())
display('Итоговая таблица состоящая из 3-х столбцов:')

,first_rec,second_rec
356,571,357
515,551,489
566,551,794
679,551,489
504,572,569


'Итоговая таблица состоящая из 3-х столбцов:'

**<font size="5" color="green"><b>ОБЩИЙ ВЫВОД</b></font>**

Из <font size="5" color="green"><b>3989</b></font> различных пар курсов, которые покупают пользователи, на каждую из <font size="5" color="green"><b>1185</b></font> пар курсов (30% от всех уникальных пар курсов) приходится одна покупка, на <font size="5" color="green"><b>578</b></font> пар (14% от всех уникальных пар курсов) - 2 покупки, <font size="5" color="green"><b>366</b></font> (9%) - 3 покупки и т.д. Самая популярная пара курсов <font size="5" color="green"><b>(551, 566)</b></font> была куплена <font size="5" color="green"><b>797</b></font> раз.

Т.к средняя частота покупок ~ 10%, где частота покупок больше 9%,  установили минимальную границу для формирования рекомендательной таблицы на уровен >=9 покупок. Самый популярный курс <font size="5" 

color="green"><b>551</b></font>. По этому  рекомендоваkb его к тем курсам, у которых частота покупки меньше 9%. Что касаемо итоговой таблицы. Был создан цикл, который выводит список рекомендаций для 

каждого курса (на основе написанной функции). Поскольку функция может вернуть 0 или 1, а не 2 рекомендации (из-за непрохождения заданного минимального порога встречаемости или отсутствия вариантов 

совместной встречаемости), цикл имеет вложенность (дополнительно проверяет условие количества полученных рекомендаций). Отсутствующие рекомендации заполняются курсами из списка самых популярных [516, 566].
